In [1]:
 import model
import dataset
import pandas as pd

import albumentations as A
from albumentations.augmentations.transforms import Flip

import torch
import pytorch_lightning as pl
from pytorch_lightning import Trainer

In [2]:
if __name__ == '__main__':
    trainer = Trainer(max_epochs = 1, gpus = 1, precision=16, amp_level='O1',deterministic=True)
    
    train_tr = A.Compose([
        A.CenterCrop(256,256,always_apply=True),
        Flip()
    ])
    
    val_tr = A.Compose([
        A.CenterCrop(256,256,always_apply=True)
    ])
    
    model = model.Classifier({'lr':3e-4,'batch_size':32,'train_tr':train_tr,'val_tr':val_tr})
    
    trainer.fit(model)
    trainer.test(model)
    out = trainer.predict(model)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.


Loaded pretrained weights for efficientnet-b3



  | Name   | Type         | Params
----------------------------------------
0 | resnet | EfficientNet | 10.7 M
----------------------------------------
10.7 M    Trainable params
0         Non-trainable params
10.7 M    Total params
42.797    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

D:\f1\ch1\model.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  prob = F.softmax(x)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_f1': 1.0, 'test_loss': 3.3453910873504356e-05}
--------------------------------------------------------------------------------


Predicting: 1250it [00:00, ?it/s]

In [12]:
import pandas as pd
def writeSub(p):
    labelmap = {0:'redbull',1:'mercedes'}
    test_df = pd.read_csv('data/sample_submission.csv')
    output_list = p.int().tolist()
    output_list = [labelmap[i] for i in output_list]
    test_df['label'] = output_list
    test_df.to_csv(path_or_buf='data/submission.csv',index = False)

In [13]:
output = torch.tensor([])

for i in range(len(out)):
    output = torch.cat((output,torch.tensor(out[i][1]).argmax(1)))

In [14]:
writeSub(output)

In [15]:
!aicrowd submission create -c f1-team-classification -f data/submission.csv

submission.csv ━━━━━━━━━━━━━━━━━━ 100.0% • 135.4/133.8 KB • 425.9 kB/s • 0:00:00
╭───────────────────── Traceback (most recent call last) ──────────────────────╮
│ /usr/local/bin/aicrowd:8 in <module>                                         │
│                                                                              │
│   5 from aicrowd.cli import cli                                              │
│   6 if __name__ == '__main__':                                               │
│   7 │   sys.argv[0] = re.sub(r'(-script\.pyw|\.exe)?$', '', sys.argv[0])     │
│ ❱ 8 │   sys.exit(cli())                                                      │
│                                                                              │
│ /usr/local/lib/python3.7/dist-packages/click/core.py:829 in __call__         │
│                                                                              │
│    826 │                                                                     │
│    827 │   def __call__(se